In [3]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Reshape, Flatten
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split

# **Configuración**
height, width, channels = 449, 593, 3
sequence_length = 10  # Número de imágenes de entrada por ventana
future_steps = 10  # Número de imágenes a predecir

# Rutas de datos
image_folder = "../data/Img_test/"  # Carpeta con imágenes
output_model = "model_lstm_vgg16_corrected.h5"  # Modelo entrenado

# **Funciones para Procesar Datos**
# Cargar y normalizar una imagen
def load_image(file_path):
    img = cv2.imread(file_path)
    img = cv2.resize(img, (width, height))  # Ajustar tamaño
    img = preprocess_input(img)  # Preprocesar para VGG16
    return img

# Preparar datos usando ventanas deslizantes
def prepare_data_sliding_window(image_paths, sequence_length, future_steps):
    images = [load_image(path) for path in image_paths]
    X, y = [], []
    for i in range(len(images) - sequence_length):
        X.append(images[i:i + sequence_length])  # Secuencia de entrada
        if i + sequence_length + future_steps <= len(images):
            y.append(images[i + sequence_length:i + sequence_length + future_steps])  # Secuencia de salida
    # Asegurar que X y y tengan el mismo tamaño
    min_length = min(len(X), len(y))
    return np.array(X[:min_length]), np.array(y[:min_length])

# Extraer características usando VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(height, width, channels))
flatten_layer = Flatten()(base_model.output)
feature_extractor = Model(inputs=base_model.input, outputs=flatten_layer)

def extract_features(image_sequences):
    num_sequences = len(image_sequences)
    features = np.zeros((num_sequences, sequence_length, feature_extractor.output_shape[1]))

    for i in range(num_sequences):
        for j in range(sequence_length):
            img = image_sequences[i, j]
            img = np.expand_dims(img, axis=0)  # Expandir dimensión del lote
            features[i, j] = feature_extractor.predict(img).squeeze()
    return features

# Cargar imágenes y preparar datos
image_paths = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(".png")])
X, y = prepare_data_sliding_window(image_paths, sequence_length, future_steps)

# Validar consistencia
if len(X) != len(y):
    raise ValueError(f"Datos inconsistentes: len(X)={len(X)}, len(y)={len(y)}.")

if len(X) == 0 or len(y) == 0:
    raise ValueError("No se encontraron suficientes datos para entrenar el modelo.")

X_features = extract_features(X)

# Dividir en entrenamiento y validación
if len(X_features) > 0 and len(y) > 0:
    X_train, X_val, y_train, y_val = train_test_split(X_features, y, test_size=0.2, random_state=42)
else:
    raise ValueError("No hay suficientes datos para entrenamiento y validación.")

# **Definir y Entrenar el Modelo**
def build_lstm_model(sequence_length, future_steps):
    model = Sequential()
    model.add(LSTM(512, input_shape=(sequence_length, feature_extractor.output_shape[1]), return_sequences=True))
    model.add(LSTM(256, return_sequences=False))
    model.add(Dense(future_steps * height * width * channels, activation='linear'))
    model.add(Reshape((future_steps, height, width, channels)))  # Salida con múltiples imágenes
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

model = build_lstm_model(sequence_length, future_steps)
model.summary()

# Callbacks
checkpoint = ModelCheckpoint(output_model, save_best_only=True, monitor="val_loss", mode="min")
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6)

# Entrenar el modelo
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[checkpoint, reduce_lr]
)

print(f"Entrenamiento completado. Modelo guardado como '{output_model}'.")


1/1 [==============================] - 0s 348ms/step


: 